# so3

> SO(3) exp and log map

In [ ]:
#| default_exp so3

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch

## Log map

The log map on SO(3) is equivalent to finding the axis-angle of rotation corresponding to a rotation matrix $\mathbf R$. This is numerically unstable near rotations $\theta = 0$ or $\theta = \pi$.

We port the numerically stable implementation for [`pytransform3d`](https://github.com/dfki-ric/pytransform3d/blob/c45e817c4a7960108afe9f5259542c8376c0e89a/pytransform3d/rotations/_conversions.py#L1719) to PyTorch to enable batched computation of multiple transformations on a GPU.

In [ ]:
#| export
def so3_log_map(R: torch.Tensor):
    """Find the axis of rotation for a rotation matrix R. The norm of the axis corresponds to the angle of rotation."""

    # Get the rotation angle
    trace = torch.einsum("...ii", R).unsqueeze(1)
    cos_angle = (trace - 1) / 2
    cos_angle = torch.clamp(cos_angle, min=-1.0, max=1.0)
    angle = cos_angle.arccos()

    # Get the rotation axis from Rodrigues' formula
    axis_unnormalized = torch.stack(
        [
            R[..., 2, 1] - R[..., 1, 2],
            R[..., 0, 2] - R[..., 2, 0],
            R[..., 1, 0] - R[..., 0, 1],
        ],
        dim=-1,
    )

    # When trace(R) is close to -1, use an alternative, which is more stable
    R_diag = torch.einsum("...ii->...i", R)
    R_diag = torch.clamp(R_diag, min=-1.0, max=1.0)
    eeT_diag = 0.5 * (R_diag + 1.0)
    signs = axis_unnormalized.sign()
    signs[signs == 0.0] = 1.0
    small_trace = eeT_diag.sqrt() * signs

    # Substitute various approximations depending on angles
    axis_unnormalized = torch.where(
        (angle - torch.pi).abs() < 1e-4,
        small_trace,
        axis_unnormalized,
    )
    axis_unnormalized = torch.where(
        angle == 0.0,
        torch.tensor([1.0, 0.0, 0.0], dtype=R.dtype, device=R.device),
        axis_unnormalized,
    )

    axis = axis_unnormalized / axis_unnormalized.norm(dim=1, keepdim=True)
    axis_angle = axis * angle
    return axis_angle

In [ ]:
#| export
def cross_product_matrix(axis_angle):
    """Convert the axis-angle to the matrix log(R)."""
    batch_size = len(axis_angle)
    log = torch.zeros(
        batch_size,
        3,
        3,
        dtype=axis_angle.dtype,
        device=axis_angle.device,
    )
    log[..., 0, 1] = -axis_angle[..., 2]
    log[..., 0, 2] = +axis_angle[..., 1]
    log[..., 1, 2] = -axis_angle[..., 0]
    log[..., 1, 0] = +axis_angle[..., 2]
    log[..., 2, 0] = -axis_angle[..., 1]
    log[..., 2, 1] = +axis_angle[..., 0]
    return log

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()